$$\frac{\partial u}{\partial t}+u\frac{\partial u}{\partial x}+v\frac{\partial u}{\partial y} = -\frac{1}{\rho}\frac{\partial p}{\partial x}+\nu \frac{\partial^2 u}{\partial x^2}+\frac{\partial^2 u}{\partial y^2}

\frac{\partial v}{\partial t}+u\\frac{\\partial v}{\\partial x}+v\\frac{\\partial v}{\\partial y} = -\\frac{1}{\\rho}\\frac{\\partial p}{\\partial y}+\\nu\\left(\\frac{\\partial^2 v}{\\partial x^2}+\\frac{\\partial^2 v}{\\partial y^2}\\right)

\\frac{\\partial^2 p}{\\partial x^2}+\\frac{\\partial^2 p}{\\partial y^2} = -\\rho\\left(\\frac{\\partial u}{\\partial x}\\frac{\\partial u}{\\partial x}+2\\frac{\\partial u}{\\partial y}\\frac{\\partial v}{\\partial x}+\\frac{\\partial v}{\\partial y}\\frac{\\partial v}{\\partial y} \\right)$


Now we will import the packages we need:

In [1]:
using MethodOfLines, DomainSets, ModelingToolkit, OrdinaryDiffEq


Now we'll construct the system:

In [7]:
@parameters t, x, y
@variables p(..), u(..), v(..)

Dt = Differential(t)
Dx = Differential(x)
Dy = Differential(y)
Dxx = Differential(x)^2
Dyy = Differential(y)^2

ρ = 1.0
ν = 0.1

eqs = [Dt(u(t,x,y)) + u(t,x,y)*Dx(u(t,x,y)) + v(t,x,y)*Dy(v(t,x,y)) ~  -(1/ρ)*Dx(p(t,x,y)) + ν*(Dxx(u(t,x,y)) + Dyy(u(t,x,y))),
       Dt(v(t,x,y)) + u(t,x,y)*Dx(v(t,x,y)) + v(t,x,y)*Dy(v(t,x,y)) ~  -(1/ρ)*Dy(p(t,x,y)) + ν*(Dxx(v(t,x,y)) + Dyy(v(t,x,y))),
       Dxx(p(t,x,y)) + Dyy(p(t,x,y)) ~  -ρ*((Dx(u(t,x,y)))^2 + 2*Dx(u(t,x,y))*Dy(v(t,x,y)) + (Dy(v(t,x,y)))^2)]

bcs = [u(0,x,y) ~ 0.0,
       v(0,x,y) ~ 0.0,
       p(0,x,y) ~ 0.0,
       u(t,x,0) ~ 0.0,
       u(t,x,2) ~ 1.0,
       v(t,x,0) ~ 0.0,
       v(t,x,2) ~ 0.0,
       u(t,0,y) ~ 0.0,
       u(t,2,y) ~ 0.0,
       v(t,0,y) ~ 0.0,
       v(t,2,y) ~ 0.0,
       Dy(p(t,x,0)) ~ 0.0,
       p(t,x,2) ~ 0.0,
       Dx(p(t,0,y)) ~ 0.0,
       Dx(p(t,2,y)) ~ 0.0]

domains = [t ∈ IntervalDomain(0.0,0.1),
           x ∈ IntervalDomain(0.0,2.0),
           y ∈ IntervalDomain(0.0,2.0)]

@named pdesys = PDESystem(eqs, bcs, domains, [t,x,y],[u(t,x,y),v(t,x,y),p(t,x,y)])

PDESystem
Equations: Equation[u(t, x, y)*Differential(x)(u(t, x, y)) + v(t, x, y)*Differential(y)(v(t, x, y)) + Differential(t)(u(t, x, y)) ~ 0.1(Differential(x)(Differential(x)(u(t, x, y))) + Differential(y)(Differential(y)(u(t, x, y)))) - Differential(x)(p(t, x, y)), u(t, x, y)*Differential(x)(v(t, x, y)) + v(t, x, y)*Differential(y)(v(t, x, y)) + Differential(t)(v(t, x, y)) ~ 0.1(Differential(x)(Differential(x)(v(t, x, y))) + Differential(y)(Differential(y)(v(t, x, y)))) - Differential(y)(p(t, x, y)), Differential(x)(Differential(x)(p(t, x, y))) + Differential(y)(Differential(y)(p(t, x, y))) ~ -(Differential(x)(u(t, x, y))^2) - (Differential(y)(v(t, x, y))^2) - 2.0Differential(x)(u(t, x, y))*Differential(y)(v(t, x, y))]
Boundary Conditions: Equation[u(0, x, y) ~ 0.0, v(0, x, y) ~ 0.0, p(0, x, y) ~ 0.0, u(t, x, 0) ~ 0.0, u(t, x, 2) ~ 1.0, v(t, x, 0) ~ 0.0, v(t, x, 2) ~ 0.0, u(t, 0, y) ~ 0.0, u(t, 2, y) ~ 0.0, v(t, 0, y) ~ 0.0, v(t, 2, y) ~ 0.0, Differential(y)(p(t, x, 0)) ~ 0.0, p(t,

Let's discretize:

In [9]:
disc = MOLFiniteDifference([x => 41, y => 41], t, advection_scheme=WENOScheme())

prob = discretize(pdesys, disc)

And Finally Solve

In [4]:
sol = solve(prob, QRBDF(), saveat=0.001)


UndefVarError: UndefVarError: `QRBDF` not defined